In [ ]:
from dotenv import load_dotenv
import json
import numpy as np
import os
import pandas as pd
import requests
import shutil

In [ ]:
load_dotenv()

folder_name = "data"
issns_file_name = "issns.xlsx"

In [ ]:
shutil.rmtree(folder_name) 
os.mkdir(folder_name)
os.remove(f"{folder_name}.zip")

In [ ]:
def getOpenAlexWorks(issn):
  # See https://github.com/ourresearch/openalex-api-tutorials/blob/main/notebooks/getting-started/paging.ipynb
  # loop through pages
  cursor = "*"
  per_page = 100
  while cursor:
      # set cursor value and request page from OpenAlex
      url = f"http://api.openalex.org/works?filter=locations.source.issn:{issn}&per_page={per_page}&cursor={cursor}"
      response = requests.get(url).json()
      # loop through partial list of results
      results = response.get("results", [])
      if cursor == "*":
        count = response.get("meta", {}).get("count")
      for result in results:
          file_name = f"{result.get('id').lower().replace('https://openalex.org/', '')}.json"
          with open(f"{folder_name}/{file_name}", "w") as file:
            json.dump(result, file, indent=4)
      # update cursor to meta.next_cursor
      cursor = response.get("meta", {}).get("next_cursor")
  return count

In [ ]:
df = pd.read_excel(issns_file_name, sheet_name="liste dedoublonnée et avis")
df = df.replace("nan", "").replace(np.nan, "")
for index, row in df.iterrows():
  issn_e = row.get("ISSN-E", "")
  issn_p = row.get("ISSN-P", "")
  issn = issn_e if issn_e and len(issn_e) > 0 else issn_p
  if len(issn) > 0:
    print(f"Collecting data for issn: {issn}")
    row["openalex_issn"] = issn
    count = getOpenAlexWorks(issn)
    row["openalex_count"] = count
  else:
    print(f"Error to retrieve issn for: {issn_e} or {issn_p}")

In [ ]:
shutil.make_archive(folder_name, "zip", folder_name)